# Explore ATT&CK Data Sources
------------------

## Goals:
* Access ATT&CK data sources in STIX format via a public TAXII server
* Learn to interact with ATT&CK data all at once
* Explore and idenfity patterns in the data retrieved
* Learn more about ATT&CK data sources

## Import ATT&CK API Client

In [4]:
from attackcti import attack_client

## Import Extra Libraries

In [5]:
# !pip install altair

In [6]:
import pandas
import numpy as np
import json

import altair as alt
alt.renderers.enable('default')

import itertools

## Initialize ATT&CK Client Class

In [7]:
lift = attack_client()

## Get All Techniques

In [8]:
all_techniques = lift.get_techniques()

In [9]:
all_techniques[0].x_mitre_domains

['enterprise-attack']

## Convert Techniques to Dataframe and Update Techniques Objects

Normalizing semi-structured JSON data into a flat table via **pandas.io.json.json_normalize**
* Reference: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

In [10]:
temp_list = []
for t in all_techniques:
    domain_name = t['x_mitre_domains'][0]
    technique_number = t['external_references'][0]['external_id']
    if 'x_mitre_data_sources' in t.keys():
        data_sources = list(set([ds.split(':')[0] for ds in t['x_mitre_data_sources']]))
        t = t.new_version(x_mitre_data_sources = data_sources)
    t = t.new_version(domain = domain_name)
    t = t.new_version(technique_id = technique_number)
    temp_list.append(json.loads(t.serialize()))
techniques = pandas.json_normalize(temp_list)
techniques.rename(columns = {'x_mitre_platforms':'platform', 'kill_chain_phases':'tactic', 'name':'technique', 'x_mitre_data_sources':'data_sources'}, inplace = True)

In [11]:
techniques = techniques.reindex(['domain','platform','tactic','technique','technique_id','data_sources'], axis=1)
techniques.head()

,domain,platform,tactic,technique,technique_id,data_sources
0,enterprise-attack,"[Linux, macOS, Windows, Network]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,"[Command, Driver, Drive, Process]"
1,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Gather Victim Host Information,T1592,[Internet Scan]
2,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Digital Certificates,T1596.003,NaN
3,enterprise-attack,"[Windows, macOS, Linux, Network]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",Keylogging,T1056.001,"[Driver, Process, Windows Registry]"
4,enterprise-attack,"[Linux, macOS, Windows]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",File/Path Exclusions,T1564.012,[File]


In [12]:
print('A total of ',len(techniques),' techniques')

A total of  858  techniques


## Techniques Per Matrix
Using **altair** python library we can start showing a few charts stacking the number of techniques with or without data sources.
Reference: https://altair-viz.github.io/

In [13]:
data = techniques
data_2 = data.groupby(['domain'])['technique'].count()
data_3 = data_2.to_frame().reset_index()
data_3

,domain,technique
0,enterprise-attack,656
1,ics-attack,83
2,mobile-attack,119


In [14]:
alt.Chart(data_3).mark_bar().encode(x='technique', y='domain', color='domain').properties(height = 200)

alt.Chart(...)

## Techniques With and Without Data Sources

In [15]:
data_source_distribution = pandas.DataFrame({
    'Techniques': ['Without DS','With DS'],
    'Count of Techniques': [techniques['data_sources'].isna().sum(),techniques['data_sources'].notna().sum()]})
bars = alt.Chart(data_source_distribution).mark_bar().encode(x='Techniques',y='Count of Techniques',color='Techniques').properties(width=200,height=300)
text = bars.mark_text(align='center',baseline='middle',dx=0,dy=-5).encode(text='Count of Techniques')
bars + text

alt.LayerChart(...)

What is the distribution of techniques based on ATT&CK Matrix?

In [16]:
data = techniques
data['Count_DS'] = data['data_sources'].str.len()
data['Ind_DS'] = np.where(data['Count_DS']>0,'With DS','Without DS')
data_2 = data.groupby(['domain','Ind_DS'])['technique'].count()
data_3 = data_2.to_frame().reset_index()
data_3

,domain,Ind_DS,technique
0,enterprise-attack,With DS,616
1,enterprise-attack,Without DS,40
2,ics-attack,With DS,69
3,ics-attack,Without DS,14
4,mobile-attack,Without DS,119


In [18]:
alt.Chart(data_3).mark_bar().encode(x='technique', y='Ind_DS').properties(height = 200)

alt.Chart(...)

What are those mitre-attack techniques without data sources?

In [19]:
data[(data['Ind_DS']=='Without DS')][0:5]

,domain,platform,tactic,technique,technique_id,data_sources,Count_DS,Ind_DS
2,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Digital Certificates,T1596.003,NaN,NaN,Without DS
8,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Purchase Technical Data,T1597.002,NaN,NaN,Without DS
17,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Artificial Intelligence,T1588.007,NaN,NaN,Without DS
45,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Employee Names,T1589.003,NaN,NaN,Without DS
54,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Code Repositories,T1593.003,NaN,NaN,Without DS


### Techniques without data sources

In [20]:
techniques_without_data_sources=techniques[techniques.data_sources.isnull()].reset_index(drop=True)

In [21]:
techniques_without_data_sources.head()

,domain,platform,tactic,technique,technique_id,data_sources,Count_DS,Ind_DS
0,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Digital Certificates,T1596.003,NaN,NaN,Without DS
1,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Purchase Technical Data,T1597.002,NaN,NaN,Without DS
2,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Artificial Intelligence,T1588.007,NaN,NaN,Without DS
3,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Employee Names,T1589.003,NaN,NaN,Without DS
4,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Code Repositories,T1593.003,NaN,NaN,Without DS


In [22]:
print('There are ',techniques['data_sources'].isna().sum(),' techniques without data sources (',"{0:.0%}".format(techniques['data_sources'].isna().sum()/len(techniques)),' of ',len(techniques),' techniques)')

There are  173  techniques without data sources ( 20%  of  858  techniques)


### Techniques With Data Sources

In [23]:
techniques_with_data_sources=techniques[techniques.data_sources.notnull()].reset_index(drop=True)

In [24]:
techniques_with_data_sources.head()

,domain,platform,tactic,technique,technique_id,data_sources,Count_DS,Ind_DS
0,enterprise-attack,"[Linux, macOS, Windows, Network]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,"[Command, Driver, Drive, Process]",4.0,With DS
1,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Gather Victim Host Information,T1592,[Internet Scan],1.0,With DS
2,enterprise-attack,"[Windows, macOS, Linux, Network]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",Keylogging,T1056.001,"[Driver, Process, Windows Registry]",3.0,With DS
3,enterprise-attack,"[Linux, macOS, Windows]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",File/Path Exclusions,T1564.012,[File],1.0,With DS
4,enterprise-attack,"[macOS, Linux]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",Linux and Mac File and Directory Permissions M...,T1222.002,"[File, Command, Process]",3.0,With DS


In [25]:
print('There are ',techniques['data_sources'].notna().sum(),' techniques with data sources (',"{0:.0%}".format(techniques['data_sources'].notna().sum()/len(techniques)),' of ',len(techniques),' techniques)')

There are  685  techniques with data sources ( 80%  of  858  techniques)


## Grouping Techniques With Data Sources By Matrix

Let's create a graph to represent the number of techniques per matrix:

In [26]:
matrix_distribution = pandas.DataFrame({
    'Domain': list(techniques_with_data_sources.groupby(['domain'])['domain'].count().keys()),
    'Count of Techniques': techniques_with_data_sources.groupby(['domain'])['domain'].count().tolist()})
bars = alt.Chart(matrix_distribution).mark_bar().encode(y='Domain',x='Count of Techniques').properties(width=300,height=100)
text = bars.mark_text(align='center',baseline='middle',dx=10,dy=0).encode(text='Count of Techniques')
bars + text

alt.LayerChart(...)

All the techniques belong to **mitre-attack** matrix which is the main **Enterprise** matrix. Reference: https://attack.mitre.org/wiki/Main_Page 

## Grouping Techniques With Data Sources by Platform

First, we need to split the **platform** column values because a technique might be mapped to more than one platform

In [27]:
techniques_platform=techniques_with_data_sources

attributes_1 = ['platform'] # In attributes we are going to indicate the name of the columns that we need to split

for a in attributes_1:
    s = techniques_platform.apply(lambda x: pandas.Series(x[a]),axis=1).stack().reset_index(level=1, drop=True)
    # "s" is going to be a column of a frame with every value of the list inside each cell of the column "a"
    s.name = a
    # We name "s" with the same name of "a".
    techniques_platform=techniques_platform.drop(a, axis=1).join(s).reset_index(drop=True)
    # We drop the column "a" from "techniques_platform", and then join "techniques_platform" with "s"

# Let's re-arrange the columns from general to specific
techniques_platform_2=techniques_platform.reindex(['domain','platform','tactic','technique','technique_id','data_sources'], axis=1)

We can now show techniques with data sources mapped to one platform at the time

In [28]:
techniques_platform_2.head()

,domain,platform,tactic,technique,technique_id,data_sources
0,enterprise-attack,Linux,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,"[Command, Driver, Drive, Process]"
1,enterprise-attack,macOS,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,"[Command, Driver, Drive, Process]"
2,enterprise-attack,Windows,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,"[Command, Driver, Drive, Process]"
3,enterprise-attack,Network,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,"[Command, Driver, Drive, Process]"
4,enterprise-attack,PRE,"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Gather Victim Host Information,T1592,[Internet Scan]


Let's create a visualization to show the number of techniques grouped by platform:

In [29]:
platform_distribution = pandas.DataFrame({
    'Platform': list(techniques_platform_2.groupby(['platform'])['platform'].count().keys()),
    'Count of Techniques': techniques_platform_2.groupby(['platform'])['platform'].count().tolist()})
bars = alt.Chart(platform_distribution,height=300).mark_bar().encode(x ='Platform',y='Count of Techniques',color='Platform').properties(width=200)
text = bars.mark_text(align='center',baseline='middle',dx=0,dy=-5).encode(text='Count of Techniques')
bars + text

alt.LayerChart(...)

In the bar chart above we can see that there are more techniques with data sources mapped to the Windows platform.

Defende-evasion and Persistence are tactics with the highest nummber of techniques with data sources

## Grouping Techniques With Data Sources by Data Source

We need to split the data source column values because a technique might be mapped to more than one data source:

In [30]:
techniques_data_source=techniques_with_data_sources

attributes_3 = ['data_sources'] # In attributes we are going to indicate the name of the columns that we need to split

for a in attributes_3:
    s = techniques_data_source.apply(lambda x: pandas.Series(x[a]),axis=1).stack().reset_index(level=1, drop=True)
    # "s" is going to be a column of a frame with every value of the list inside each cell of the column "a"
    s.name = a
    # We name "s" with the same name of "a".
    techniques_data_source = techniques_data_source.drop(a, axis=1).join(s).reset_index(drop=True)
    # We drop the column "a" from "techniques_data_source", and then join "techniques_data_source" with "s"

# Let's re-arrange the columns from general to specific
techniques_data_source_2 = techniques_data_source.reindex(['domain','platform','tactic','technique','technique_id','data_sources'], axis=1)

# We are going to edit some names inside the dataframe to improve the consistency:
techniques_data_source_3 = techniques_data_source_2.replace(['Process monitoring','Application logs'],['Process Monitoring','Application Logs'])

We can now show techniques with data sources mapped to one data source at the time

In [31]:
techniques_data_source_3.head()

,domain,platform,tactic,technique,technique_id,data_sources
0,enterprise-attack,"[Linux, macOS, Windows, Network]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,Command
1,enterprise-attack,"[Linux, macOS, Windows, Network]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,Driver
2,enterprise-attack,"[Linux, macOS, Windows, Network]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,Drive
3,enterprise-attack,"[Linux, macOS, Windows, Network]","[{'kill_chain_name': 'mitre-attack', 'phase_na...",Disk Structure Wipe,T1561.002,Process
4,enterprise-attack,[PRE],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",Gather Victim Host Information,T1592,Internet Scan


Let's create a visualization to show the number of techniques grouped by data sources:

In [32]:
data_source_distribution = pandas.DataFrame({
    'Data Source': list(techniques_data_source_3.groupby(['data_sources'])['data_sources'].count().keys()),
    'Count of Techniques': techniques_data_source_3.groupby(['data_sources'])['data_sources'].count().tolist()})
bars = alt.Chart(data_source_distribution,width=800,height=300).mark_bar().encode(x ='Data Source',y='Count of Techniques',color='Data Source').properties(width=1200)
text = bars.mark_text(align='center',baseline='middle',dx=0,dy=-5).encode(text='Count of Techniques')
bars + text

alt.LayerChart(...)

A few interesting things from the bar chart above:
* Process Monitoring, File Monitoring, and Process Command-line parameters are the Data Sources with the highest number of techniques
* There are some data source names that include string references to Windows such as PowerShell, Windows and wmi

## Most Relevant Groups Of Data Sources Per Technique

### Number Of Data Sources Per Technique

Although identifying the data sources with the highest number of techniques is a good start, they usually do not work alone. You might be collecting **Process Monitoring** already but you might be still missing a lot of context from a data perspective.

In [33]:
data_source_distribution_2 = pandas.DataFrame({
    'Techniques': list(techniques_data_source_3.groupby(['technique'])['technique'].count().keys()),
    'Count of Data Sources': techniques_data_source_3.groupby(['technique'])['technique'].count().tolist()})

data_source_distribution_3 = pandas.DataFrame({
    'Number of Data Sources': list(data_source_distribution_2.groupby(['Count of Data Sources'])['Count of Data Sources'].count().keys()),
    'Count of Techniques': data_source_distribution_2.groupby(['Count of Data Sources'])['Count of Data Sources'].count().tolist()})

bars = alt.Chart(data_source_distribution_3).mark_bar().encode(x ='Number of Data Sources',y='Count of Techniques').properties(width=500)
text = bars.mark_text(align='center',baseline='middle',dx=0,dy=-5).encode(text='Count of Techniques')
bars + text

alt.LayerChart(...)

The image above shows you the number data sources needed per techniques according to ATT&CK:
* There are 71 techniques that require 3 data sources as enough context to validate the detection of them according to ATT&CK
* Only one technique has 12 data sources
* One data source only applies to 19 techniques

Let's create subsets of data sources with the data source column defining and using a python function:

In [34]:
# https://stackoverflow.com/questions/26332412/python-recursive-function-to-display-all-subsets-of-given-set
def subs(l):
    res = []
    for i in range(1, len(l) + 1):
        for combo in itertools.combinations(l, i):
            res.append(list(combo))
    return res

Before applying the function, we need to use lowercase data sources names and sort data sources names to improve consistency:

In [35]:
df = techniques_with_data_sources[['data_sources']]

In [36]:
for index, row in df.iterrows():
    row["data_sources"]=[x.lower() for x in row["data_sources"]]
    row["data_sources"].sort()

In [37]:
df.head()

,data_sources
0,"[command, drive, driver, process]"
1,[internet scan]
2,"[driver, process, windows registry]"
3,[file]
4,"[command, file, process]"


Let's apply the function and split the subsets column:

In [38]:
df['subsets']=df['data_sources'].apply(subs)

/var/folders/9z/8xhqw8x1611fcbhzl339yrs40000gn/T/ipykernel_19687/1660526906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subsets']=df['data_sources'].apply(subs)


In [39]:
df.head()

,data_sources,subsets
0,"[command, drive, driver, process]","[[command], [drive], [driver], [process], [com..."
1,[internet scan],[[internet scan]]
2,"[driver, process, windows registry]","[[driver], [process], [windows registry], [dri..."
3,[file],[[file]]
4,"[command, file, process]","[[command], [file], [process], [command, file]..."


We need to split the subsets column values:

In [40]:
techniques_with_data_sources_preview = df

In [41]:
attributes_4 = ['subsets']

for a in attributes_4:
    s = techniques_with_data_sources_preview.apply(lambda x: pandas.Series(x[a]),axis=1).stack().reset_index(level=1, drop=True)
    s.name = a
    techniques_with_data_sources_preview = techniques_with_data_sources_preview.drop(a, axis=1).join(s).reset_index(drop=True)
    
techniques_with_data_sources_subsets = techniques_with_data_sources_preview.reindex(['data_sources','subsets'], axis=1)


In [42]:
techniques_with_data_sources_subsets.head()

,data_sources,subsets
0,"[command, drive, driver, process]",[command]
1,"[command, drive, driver, process]",[drive]
2,"[command, drive, driver, process]",[driver]
3,"[command, drive, driver, process]",[process]
4,"[command, drive, driver, process]","[command, drive]"


Let's add three columns to analyse the dataframe: subsets_name (Changing Lists to Strings), subsets_number_elements ( Number of data sources per subset) and number_data_sources_per_technique

In [43]:
techniques_with_data_sources_subsets['subsets_name']=techniques_with_data_sources_subsets['subsets'].apply(lambda x: ','.join(map(str, x)))
techniques_with_data_sources_subsets['subsets_number_elements']=techniques_with_data_sources_subsets['subsets'].str.len()
techniques_with_data_sources_subsets['number_data_sources_per_technique']=techniques_with_data_sources_subsets['data_sources'].str.len()

In [44]:
techniques_with_data_sources_subsets.head()

,data_sources,subsets,subsets_name,subsets_number_elements,number_data_sources_per_technique
0,"[command, drive, driver, process]",[command],command,1,4
1,"[command, drive, driver, process]",[drive],drive,1,4
2,"[command, drive, driver, process]",[driver],driver,1,4
3,"[command, drive, driver, process]",[process],process,1,4
4,"[command, drive, driver, process]","[command, drive]","command,drive",2,4


As it was described above, we need to find grups pf data sources, so we are going to filter out all the subsets with only one data source:

In [45]:
subsets = techniques_with_data_sources_subsets

subsets_ok=subsets[subsets.subsets_number_elements != 1]

In [46]:
subsets_ok.head()

,data_sources,subsets,subsets_name,subsets_number_elements,number_data_sources_per_technique
4,"[command, drive, driver, process]","[command, drive]","command,drive",2,4
5,"[command, drive, driver, process]","[command, driver]","command,driver",2,4
6,"[command, drive, driver, process]","[command, process]","command,process",2,4
7,"[command, drive, driver, process]","[drive, driver]","drive,driver",2,4
8,"[command, drive, driver, process]","[drive, process]","drive,process",2,4


Finally, we calculate the most relevant groups of data sources (Top 15):

In [47]:
subsets_graph = subsets_ok.groupby(['subsets_name'])['subsets_name'].count().to_frame(name='subsets_count').sort_values(by='subsets_count',ascending=False)[0:15]

In [48]:
subsets_graph

,subsets_count
subsets_name,
"command,process",235
"command,file",155
"file,process",149
"command,file,process",111
"process,windows registry",73
"command,windows registry",71
"application log,network traffic",64
"command,process,windows registry",63
"network traffic,process",59


In [49]:
subsets_graph_2 = pandas.DataFrame({
    'Data Sources': list(subsets_graph.index),
    'Count of Techniques': subsets_graph['subsets_count'].tolist()})

bars = alt.Chart(subsets_graph_2).mark_bar().encode(x ='Data Sources', y ='Count of Techniques', color='Data Sources').properties(width=500)
text = bars.mark_text(align='center',baseline='middle',dx= 0,dy=-5).encode(text='Count of Techniques')
bars + text

alt.LayerChart(...)

Group (Process Monitoring - Process Command-line parameters) is the is the group of data sources with the highest number of techniques. This group of data sources are suggested to hunt 78 techniques

## Let's Split all the Information About Techniques With Data Sources Defined: Matrix, Platform, Tactic and Data Source

Let's split all the relevant columns of the dataframe:

In [50]:
techniques_data = techniques_with_data_sources

attributes = ['platform','tactic','data_sources'] # In attributes we are going to indicate the name of the columns that we need to split

for a in attributes:
    s = techniques_data.apply(lambda x: pandas.Series(x[a]),axis=1).stack().reset_index(level=1, drop=True)
    # "s" is going to be a column of a frame with every value of the list inside each cell of the column "a"
    s.name = a
    # We name "s" with the same name of "a".
    techniques_data=techniques_data.drop(a, axis=1).join(s).reset_index(drop=True)
    # We drop the column "a" from "techniques_data", and then join "techniques_data" with "s"

# Let's re-arrange the columns from general to specific
techniques_data_2=techniques_data.reindex(['domain','platform','tactic','technique','technique_id','data_sources'], axis=1)

# We are going to edit some names inside the dataframe to improve the consistency:
techniques_data_3 = techniques_data_2.replace(['Process monitoring','Application logs'],['Process Monitoring','Application Logs'])

techniques_data_3.head()

,domain,platform,tactic,technique,technique_id,data_sources
0,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Command
1,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Driver
2,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Drive
3,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Process
4,enterprise-attack,macOS,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Command


Do you remember data sources names with a reference to Windows? After splitting the dataframe by platforms, tactics and data sources, are there any macOC or linux techniques that consider windows data sources? Let's identify those rows:

In [51]:
# After splitting the rows of the dataframe, there are some values that relate windows data sources with platforms like linux and masOS.
# We need to identify those rows
conditions = [(techniques_data_3['platform']=='Linux')&(techniques_data_3['data_sources'].str.contains('windows',case=False)== True),
             (techniques_data_3['platform']=='macOS')&(techniques_data_3['data_sources'].str.contains('windows',case=False)== True),
             (techniques_data_3['platform']=='Linux')&(techniques_data_3['data_sources'].str.contains('powershell',case=False)== True),
             (techniques_data_3['platform']=='macOS')&(techniques_data_3['data_sources'].str.contains('powershell',case=False)== True),
             (techniques_data_3['platform']=='Linux')&(techniques_data_3['data_sources'].str.contains('wmi',case=False)== True),
             (techniques_data_3['platform']=='macOS')&(techniques_data_3['data_sources'].str.contains('wmi',case=False)== True)]
# In conditions we indicate a logical test

choices = ['NO OK','NO OK','NO OK','NO OK','NO OK','NO OK']
# In choices, we indicate the result when the logical test is true

techniques_data_3['Validation'] = np.select(conditions,choices,default='OK')
# We add a column "Validation" to "techniques_data_3" with the result of the logical test. The default value is going to be "OK"

What is the inconsistent data?

In [52]:
techniques_analysis_data_no_ok = techniques_data_3[techniques_data_3.Validation == 'NO OK']
# Finally, we are filtering all the values with NO OK

techniques_analysis_data_no_ok.head()

,domain,platform,tactic,technique,technique_id,data_sources,Validation
25,enterprise-attack,macOS,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Keylogging,T1056.001,Windows Registry,NO OK
28,enterprise-attack,macOS,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Keylogging,T1056.001,Windows Registry,NO OK
31,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Keylogging,T1056.001,Windows Registry,NO OK
34,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Keylogging,T1056.001,Windows Registry,NO OK
78,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",OS Credential Dumping,T1003,Windows Registry,NO OK


In [53]:
print('There are ',len(techniques_analysis_data_no_ok),' rows with inconsistent data')

There are  117  rows with inconsistent data


What is the impact of this inconsistent data from a platform and data sources perspective?

In [54]:
df = techniques_with_data_sources

attributes = ['platform','data_sources']

for a in attributes:
    s = df.apply(lambda x: pandas.Series(x[a]),axis=1).stack().reset_index(level=1, drop=True)
    s.name = a
    df=df.drop(a, axis=1).join(s).reset_index(drop=True)
    
df_2=df.reindex(['domain','platform','tactic','technique','technique_id','data_sources'], axis=1)
df_3 = df_2.replace(['Process monitoring','Application logs'],['Process Monitoring','Application Logs'])

conditions = [(df_3['data_sources'].str.contains('windows',case=False)== True),
              (df_3['data_sources'].str.contains('powershell',case=False)== True),
              (df_3['data_sources'].str.contains('wmi',case=False)== True)]

choices = ['Windows','Windows','Windows']

df_3['Validation'] = np.select(conditions,choices,default='Other')
df_3['Num_Tech'] = 1
df_4 = df_3[df_3.Validation == 'Windows']
df_5 = df_4.groupby(['data_sources','platform'])['technique'].nunique()
df_6 = df_5.to_frame().reset_index()

In [55]:
alt.Chart(df_6).mark_bar().encode(x=alt.X('technique', stack="normalize"),    y='data_sources',    color='platform').properties(height=200)

alt.Chart(...)

There are techniques that consider Windows Error Reporting, Windows Registry, and Windows event logs as data sources and they also consider platforms like Linux and masOS. We do not need to consider this rows because those data sources can only be managed at a Windows environment. These are the techniques that we should not consider in our data base:

In [56]:
techniques_analysis_data_no_ok[['technique','data_sources']].drop_duplicates().sort_values(by='data_sources',ascending=True)

,technique,data_sources
3451,Remote Services,WMI
250,Fileless Storage,WMI
4865,Obfuscated Files or Information,WMI
4637,Event Triggered Execution,WMI
25,Keylogging,Windows Registry
3497,Disable or Modify System Firewall,Windows Registry
3639,Browser Extensions,Windows Registry
3870,Impair Defenses,Windows Registry
4109,Clear Network Connection History and Configura...,Windows Registry
4264,Install Root Certificate,Windows Registry


Without considering this inconsistent data, the final dataframe is:

In [57]:
techniques_analysis_data_ok = techniques_data_3[techniques_data_3.Validation == 'OK']
techniques_analysis_data_ok.head()

,domain,platform,tactic,technique,technique_id,data_sources,Validation
0,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Command,OK
1,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Driver,OK
2,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Drive,OK
3,enterprise-attack,Linux,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Process,OK
4,enterprise-attack,macOS,"{'kill_chain_name': 'mitre-attack', 'phase_nam...",Disk Structure Wipe,T1561.002,Command,OK


In [58]:
print('There are ',len(techniques_analysis_data_ok),' rows of data that you can play with')

There are  6145  rows of data that you can play with


## Getting Techniques by Data Sources

This function gets techniques' information that includes specific data sources

In [59]:
data_source = 'PROCESS'

In [60]:
results = lift.get_techniques_by_data_sources(data_source)

In [61]:
len(results)

332

In [62]:
type(results)

list

In [63]:
results2 = lift.get_techniques_by_data_sources('pRoceSS','commAnd')

In [64]:
len(results2)

398

In [65]:
results2[1]

AttackPattern(type='attack-pattern', spec_version='2.1', id='attack-pattern--09a60ea3-a8d1-4ae5-976e-5783248b72a4', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-02-11T18:58:11.791Z', modified='2023-10-01T14:01:12.167Z', name='Keylogging', description='Adversaries may log user keystrokes to intercept credentials as the user types them. Keylogging is likely to be used to acquire credentials for new access opportunities when [OS Credential Dumping](https://attack.mitre.org/techniques/T1003) efforts are not effective, and may require an adversary to intercept keystrokes on a system for a substantial period of time before credentials can be successfully captured. In order to increase the likelihood of capturing credentials quickly, an adversary may also perform actions such as clearing browser cookies to force users to reauthenticate to systems.(Citation: Talos Kimsuky Nov 2021)\n\nKeylogging is the most prevalent type of input capture, with many different 